# Cleaning DoD Comptroller Budget Data

#### This pulls the main 2019 excel budget spreadsheets from the comptroller website, cleans, and concatenates them to get the 2017 data
#### This provides some of the picture around agency budgets, but there are particular limits when looking at non-Service budgets

### Loading Packages

In [2]:
import pandas as pd
import os
import numpy as np
import requests

#this will put the files in whatever folder you're in

### Pulling Files From Comptroller Site

In [ ]:
def pullFile(year, cleanLink):
    r = requests.get(cleanLink, allow_redirects=True)
    name=str(year)+cleanLink.split("/")[-1]
    open(name, 'wb').write(r.content)

def pullBudgetFiles(year):
    filesWeWant=["c1", "m1", "p1", "r1", "rf1", "o1"]  
    for file in filesWeWant:
        cleanLink=f'https://comptroller.defense.gov/Portals/45/Documents/defbudget/fy{year}/{file}.xlsx'
        pullFile(year, cleanLink)

def pullDataRange(firstYear, lastYear):
    for year in range(firstYear, lastYear+1):
        pullBudgetFiles(year)

os.chdir(r'H:\_MyComputer\Documents\Git Repos\budgetData\concatenatingBudgetData')
pullDataRange(2020, 2020)

### Reading Data Files

In [3]:
def cleanColumnName(columnName):
    noExits=columnName.replace('\n'," ")
    lessWhiteSpace=' '.join(' '.join(noExits.split()).split())
    return(lessWhiteSpace)

def aggData(budgetFiles, year):
    allBudgetData=pd.DataFrame()
    for spreadsheet in budgetFiles:
        filename = os.path.join(os.getcwd(), spreadsheet)
        df = pd.read_excel(filename, None, skiprows=1)
    #print(df.keys())
        for tabName in df.keys():
            df[tabName].columns=list(map(cleanColumnName,df[tabName].columns))
            df[tabName]['fileName']=spreadsheet.split(".")[0]
            df[tabName]['tabName']=tabName
            if str(year) in tabName:
                print(tabName)
                #print(spreadsheet.split(".")[0], df[tabName].columns)
        df['budgetAll']=allBudgetData
        #allBudgetData=pd.concat(df.values(), sort=True)
        allBudgetData=pd.concat(df.values())
        allBudgetData['Year']=str(year)
    return(allBudgetData)

def pullABunchOfYears():
    df=pd.DataFrame()
    listOfYears=[i for i in range(2013, 2020)]
    for year in listOfYears:
        budgetFiles=[i for i in os.listdir() if i.lower().startswith(str(year))]
        yearData=aggData(budgetFiles, year)
        df=pd.concat([df, yearData])
    return(df)
#allData=aggData(budgetFiles)
#print(allData.count().head())
df=pullABunchOfYears()


FY 2013
FY 2013 M1
FY 2013 Base
FY 2013 OCO
FY 2013 P1
FY 2013 Base
FY 2013 OCO
FY 2013 R1
FY 2013 Base
FY 2013 OCO
FY 2013 RF-1
FY 2013 Base RF-1
FY 2013 OCO RF-1
FY 2013 O1
FY 2013 Base O1
FY 2013 OCO O1
FY 2014
FY 2014 Base
FY 2014 Base
FY 2014 Base
FY 2014 Base
FY 2014 Base
FY 2015
FY 2015 Base
FY 2015 Base
FY 2015 Base
FY 2015 Base
FY 2015 Base
FY 2016
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2016 Base
FY 2016 OCO
FY 2016 Total
FY 2017
FY 2017 OCO
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2017 Base
FY 2017 OCO
FY 2017 Total
FY 2018
FY 2018 OCO
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2018 Base
FY 2018 OCO
FY 2018 Total
FY 2019
 FY 201

### Get 2017 Tabs and Values

In [4]:
def getYearData(df):
    data2017=df.loc[df['tabName'].str.contains("2017")]
    keepColumns=[i for i in data2017.columns if "2018" not in i and "2019" not in i]
    data2017=data2017[keepColumns].dropna(axis='columns',how='all')
    return(data2017)

data2017=getYearData(allData)
print(pd.crosstab(data2017['tabName'], data2017['fileName']))



NameError: name 'allData' is not defined

### Standardize Budget Values

In [21]:
def genBudgetVars():
    dictOfCostVar={"c1": 'TOA Amount', "m1": 'FY 2017 (Base + OCO)', 
                   "o1": 'FY 2017 (Base + OCO)', "p1": "FY 2017 (Base + OCO) Amount",
                   "p1r": "FY 2017 (Base + OCO) Amount", "r1": "FY 2017 (Base + OCO)", "rf1": "FY 2017 (Base + OCO)"}
    return(dictOfCostVar)

def cleanBudgetVar(dictOfCostVar, df):
    df['2017 Budget Amount']=0
    for key in dictOfCostVar.keys():
        #print(key)
        #print(len(df.loc[df['fileName']==key]))
        #print(df.loc[df['fileName']==key][dictOfCostVar[key]].sum())
        df.loc[df['fileName']==key,'2017 Budget Amount']=df.loc[df['fileName']==key]['2017 Budget Amount']+df.loc[df['fileName']==key][dictOfCostVar[key]]
    return(df)

dictOfCostVar=genBudgetVars()
df=cleanBudgetVar(dictOfCostVar, data2017)



### Accuracy Check

In [32]:
"${0:,.0f}".format(df['2017 Budget Amount'].sum())

'$618,157,868'

In [46]:
df['Organization'].value_counts()

NAVY     877
ARMY     737
AF       656
OSD       87
SOCOM     63
MDA       50
DHA       43
DLA       32
DISA      28
DARPA     17
DEFW      17
NSA       15
DTRA      11
TJS       10
CBDP      10
WHS        8
DODEA      8
DSCA       6
DIA        5
DSS        4
DHRA       4
IG         3
DCMA       3
OTE        3
NGA        2
DCAA       2
DTIC       2
DMACT      2
DECA       1
CMP        1
DPAA       1
DTSA       1
OEA        1
DFAS       1
DLSA       1
CAAF       1
DAU        1
Name: Organization, dtype: int64

In [33]:
def summarizeOrgInfo(df, org):
    print(f'Organization: {org}')
    budgetLines=(df.loc[df['Organization']==org])
    print(f'This has {len(budgetLines)} lines in the 2017 budget.')
    if len(budgetLines)>0:
        spendCats=budgetLines['fileName'].unique()
        print(f'They are from the following categories: {spendCats}.')
        print(f"Spending is as follows:")
        print(f"{budgetLines[['fileName','2017 Budget Amount']].groupby(['fileName']).sum()}.")
        print(f"Total budget is {budgetLines['2017 Budget Amount'].sum()}.")
              
              
    print("")

Organizations=list(set(df['Organization']))
for org in Organizations:
    summarizeOrgInfo(df, org)
    
    


Organization: nan
This has 0 lines in the 2017 budget.

Organization: OEA
This has 1 lines in the 2017 budget.
They are from the following categories: ['o1'].
Spending is as follows:
          2017 Budget Amount
fileName                    
o1                  311785.0.
Total budget is 311785.0.

Organization: OSD
This has 87 lines in the 2017 budget.
They are from the following categories: ['o1' 'r1' 'p1' 'c1'].
Spending is as follows:
          2017 Budget Amount
fileName                    
c1                      18.0
o1                 2122282.0
p1                  138289.0
r1                 4084372.0.
Total budget is 6344961.0.

Organization: DLSA
This has 1 lines in the 2017 budget.
They are from the following categories: ['o1'].
Spending is as follows:
          2017 Budget Amount
fileName                    
o1                  135896.0.
Total budget is 135896.0.

Organization: DEFW
This has 17 lines in the 2017 budget.
They are from the following categories: ['o1' 'p1' 'c1']

In [5]:
df.columns

Index(['AG / BSA', 'AG / Budget SubActivity (BSA) Title', 'AG Title',
       'Account', 'Account Short Title', 'Account Title', 'Add/ Non-Add',
       'Appn Amount', 'Auth Amount', 'Auth For Appn Amount',
       ...
       'SAG Title', 'State Country', 'State Country Title', 'TOA Amount',
       'TOA Base Request with CR Adj *',
       'TOA Total Base Request with CR Adj *', 'Treasury Agency', 'Year',
       'fileName', 'tabName'],
      dtype='object', length=238)

In [9]:
df.loc[df['Organization']=="DISA"].to_excel("DISA.xlsx")